In [1]:
!pip install -q gdown

In [2]:
#drive_paths = ['https://drive.google.com/file/d/1naOatlOMvDhFhEXEcA_Q9yNey6pSgxeA/view?usp=sharing',
#               'https://drive.google.com/file/d/1NvF6nqXnLsTmAzshnU2rO8WnMb6RuF8y/view?usp=sharing',
#               'https://drive.google.com/file/d/1et2sdK2Isd6YG3x-A9I9ls_JO2RY6hRt/view?usp=sharing',
#               'https://drive.google.com/file/d/1qNFtljOBWefyIDIozvaD4T24TDiPsOmG/view?usp=sharing',
#               'https://drive.google.com/file/d/182FFFY0vj4c7QUSE4syY5nCg5EOZ63AL/view?usp=sharing']


#FINE 


drive_paths = ['https://drive.google.com/file/d/1G6o7NGiQq9HJbupCbWfk9feeI3Mn8kSC/view?usp=sharing',
               'https://drive.google.com/file/d/1evSe5N_1I60llmQ1BDYxr6LeTohataYo/view?usp=sharing',
               'https://drive.google.com/file/d/19gPPwAe94Wa38zWq1bIELQIYS2Y8Dsyx/view?usp=sharing',
               'https://drive.google.com/file/d/1EHDHSStMUQW49RRcB4n4R0DSuLXtxhyO/view?usp=sharing',
               'https://drive.google.com/file/d/11pPTW3YsVyazLi7s7SVvkvxI1WXEkHO6/view?usp=sharing']

In [3]:
import gdown


for i, path in enumerate(drive_paths):
    path = path.split('/')[-2]
    gdown.download(f'https://drive.google.com/uc?id={path}', f'fold-{i}.h5', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1G6o7NGiQq9HJbupCbWfk9feeI3Mn8kSC
To: /kaggle/working/fold-0.h5
213MB [00:00, 300MB/s]
Downloading...
From: https://drive.google.com/uc?id=1evSe5N_1I60llmQ1BDYxr6LeTohataYo
To: /kaggle/working/fold-1.h5
213MB [00:00, 290MB/s]
Downloading...
From: https://drive.google.com/uc?id=19gPPwAe94Wa38zWq1bIELQIYS2Y8Dsyx
To: /kaggle/working/fold-2.h5
213MB [00:00, 267MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EHDHSStMUQW49RRcB4n4R0DSuLXtxhyO
To: /kaggle/working/fold-3.h5
213MB [00:00, 332MB/s]
Downloading...
From: https://drive.google.com/uc?id=11pPTW3YsVyazLi7s7SVvkvxI1WXEkHO6
To: /kaggle/working/fold-4.h5
213MB [00:00, 249MB/s]


In [4]:
!pip install -q efficientnet >> /dev/null

import random, os
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf, math
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import glob
from scipy.stats import rankdata

import tensorflow_hub as tfhub

In [5]:
DEVICE = 'TPU'

if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
#strategy = auto_select_accelerator()
BATCH_SIZE = 16
batch_size = 16


MODEL_GCS_PATH = 'gs://kds-07646ef73491847b90e9814314e77b345c1a6feb386e46b81a3598b9/tfhub_models/efficientnetv2-m-21k-ft1k/feature_vector'
#'gs://kds-a4518d8bb2ac917f269f913bed4066c2943089632d96cf03f86b5699/tfhub_models/efficientnetv2-m-21k-ft1k/feature_vector'






EFF_NET = 3
img_size = (672, 672)
weights = 'noisy-student'



GCS_PATH = KaggleDatasets().get_gcs_path('testsetitfrecords0')
files_all = np.sort(np.array(tf.io.gfile.glob([GCS_PATH + '/test_seti.tfrec'])))

connecting to TPU...
Running on TPU  grpc://10.0.0.2:8470
initializing  TPU ...
TPU initialized
REPLICAS: 8


In [6]:
sub = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
#sub['path'] = sub['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

In [7]:
def prepare_image(img, augment=False, dim=256):    
    img = tf.io.decode_raw(img, tf.float16)
    img = tf.reshape(img, [273, 256, 3])
    img = tf.cast(img, tf.float32)
    
    img = tf.clip_by_value(img, clip_value_min=-6.0, clip_value_max=6.0)
    img = img / 3.0
    
    #img = tf.image.resize(img, [img_resize[0], img_resize[1]])
    
    img = tf.unstack(img, axis = 2)
    img = tf.concat(img, axis = 0)
    img = tf.expand_dims(img, 2)
    img = tf.image.grayscale_to_rgb(img)
    
    img = tf.image.resize(img, [img_size[0], img_size[1]])
    
    img = tf.image.flip_up_down(img)
    img = tf.image.flip_left_right(img)

                      
    img = tf.reshape(img, [dim[0], dim[1], 3])
            
    return img

def read_labeled_tfrecord(example):
    tfrec_format = {
        'signal'   : tf.io.FixedLenFeature([], tf.string),
        'id'       : tf.io.FixedLenFeature([], tf.string),
        'target'   : tf.io.FixedLenFeature([], tf.int64),
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['signal'], example['target']

def get_dataset(files, augment = False, shuffle = False, repeat = False, 
                labeled=True, return_image_names=True, batch_size=16, dim=256):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)    
    #ds = ds.map(lambda example: read_labeled_tfrecord(example, return_image_names), 
    #                num_parallel_calls=AUTO)    
    
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=False, dim=dim), 
                                               imgname_or_label), num_parallel_calls=AUTO)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)

        
    return ds

dtest = get_dataset(files_all, dim = img_size, batch_size = batch_size)

#test_decoder = build_decoder(dim = img_size, flip = False)
#test_decoder_flip = build_decoder(flip = True)

#dtest = build_dataset(sub['path'], test_decoder)
#dtest_flip = build_dataset(sub['path'], test_decoder_flip)

#oof_names.append( np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())]))

In [8]:
#a =  np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(dtest.unbatch())])

In [9]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6]

def build_model(dim=128, ef=0):
    
    
    #inp = tf.keras.layers.Input(shape=(dim[0],dim[1],3))
    #base = EFNS[ef](input_shape=(dim[0],dim[1],3), weights=weights,include_top=False)
    #x = base(inp)
    #x = tf.keras.layers.GlobalAveragePooling2D()(x)
    #x = tf.keras.layers.Dropout(0.25)(x)
    #x = tf.keras.layers.Dense(1,activation='sigmoid')(x)
    #model = tf.keras.Model(inputs=inp,outputs=x)


    model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=[dim[0],dim[1],3]),
            tfhub.KerasLayer(MODEL_GCS_PATH , trainable=True),
            tf.keras.layers.Dropout(rate=0.25),
            tf.keras.layers.Dense(1,activation='sigmoid')
        ])
    
    
    #opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    #loss = tf.keras.losses.BinaryCrossentropy() #label_smoothing=0.05
    #model.compile(optimizer=opt,loss=loss,metrics=['AUC'])
    return model

In [10]:
with strategy.scope():
    
    m0 = build_model(dim=img_size ,ef=EFF_NET)
    m1 = build_model(dim=img_size ,ef=EFF_NET)
    m2 = build_model(dim=img_size ,ef=EFF_NET)
    m3 = build_model(dim=img_size ,ef=EFF_NET)
    m4 = build_model(dim=img_size ,ef=EFF_NET)
    
    m0.load_weights('fold-0.h5')
    m1.load_weights('fold-1.h5')
    m2.load_weights('fold-2.h5')
    m3.load_weights('fold-3.h5')
    m4.load_weights('fold-4.h5')
    
    y0 = m0.predict(dtest,verbose=1)
    y1 = m1.predict(dtest,verbose=1)
    y2 = m2.predict(dtest,verbose=1)
    y3 = m3.predict(dtest,verbose=1)
    y4 = m4.predict(dtest,verbose=1)

313/313 [==============================] - 83s 202ms/step


In [11]:
raw = y0  + y1 + y2 + y3 + y4
raw = raw / 5.0
np.save('raw.npy', raw)

In [12]:
y0 = rankdata(y0, axis= 0)
y1 = rankdata(y1, axis= 0)
y2 = rankdata(y2, axis= 0)
y3 = rankdata(y3, axis= 0)
y4 = rankdata(y4, axis= 0)

In [13]:
preds = y0  + y1 + y2 + y3 + y4

In [14]:
preds = preds / (preds.shape[0] * 1.0)

In [15]:
preds.shape

(39995, 1)

In [16]:
sub['target'] = preds

In [17]:
sub.to_csv('submission.csv', index = False)